In [50]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

from scipy.stats import norm
from scipy.stats import t
from scipy.stats import chi2

# Átomo de Hidrogeno y Deuterio.

In [51]:
def Minimos_cuadrados(x,y,n=1):
    l = x.shape[0]
    b = y
    
    A = np.ones((l,n+1))
    
    for i in range(1,n+1):
        A[:,i] = x**i
        
    AT = np.dot(A.T,A)
    bT = np.dot(A.T,b)

    xsol = np.linalg.solve(AT,bT)
    
    return xsol

def GetModel(x,p):
    
    y = 0.
    for i in range(len(p)):
        y += p[i]*x**i
        
    return y

def GetError(x,y,p,n=1):
    
    l = x.shape[0]
    
    A = np.ones((l,n+1))
    
    for i in range(1,n+1):
        A[:,i] = x**i
        
    # Residuos
    R = y - np.dot(A,p)
    
    sigma2 = np.dot(R.T,R)/(len(y)-len(p))
    
    Cov = sigma2*np.linalg.inv(np.dot(A.T,A))
    
    R2 = None
    if np.shape(Cov) == (2,2):
        R2 = (Cov[0][1]**2)/(Cov[0][0]*Cov[1][1])

    return (Cov,R2,R)

def Get_sigma(Cov):
    sigmaai = np.sqrt(np.diagonal(Cov))
    return sigmaai[::-1]

def Error_relativo(X_real,X_exp):
    return (np.abs(X_exp - X_real)/X_real)

def presicion(d,v):
    p = (d/v)
    if p <= 0.1:
        return p,True
    else:
        return p,False
    
def exactitud(xr,xm,d):
    e = (np.abs(xr-xm))
    if e <= (2.5*d):
        return e,True
    else:
        return e,False

X = sym.Symbol('x',real=True)
Y = sym.Symbol('y',real=True)

In [52]:
c = 299798458*1e09 #nm/s
me = 0.51099895*1e06/(c**2) #eV/c**2
mp = 938.27208816*1e06/(c**2) #eV/c**2
mn = 939.56542052*1e06/(c**2) #eV/c**2
h = 4.136667696*1e-15 #eV*s
h_barra = 6.582119569*1e-16 #eV*s
e_0 = 55.26349406*1e-03 #e**2/(cV*nm)
e = 1 #e
a_0 = 0.0529177210903 #nm
Ry = 13.605693122994 #eV

def Mr(m1=me,m2=mp):
    return (m1*m2)/(m1+m2)

def coef_Mr(M,m):
    return M/(m+M)

def LongOnda(Z=1,M=mp,m=me,ni=3,nf=2):
    return ((h*c)/(Ry*(Z**2)*coef_Mr(M,m)))*(((1/(nf**2)) - (1/(ni**2)))**(-1))

H_a = LongOnda()
D_a = LongOnda(1,mp+mn,me,3,2)
d = np.abs(H_a-D_a)
d

0.17883481948592816

# Datos.

In [53]:
d_Ne_H = 1028 #píxeles
d_H_D = 61 #píxeles
Ne_ = 653.28824 #nm
H_a = 656.2795 #nm
D_a = 656.1012 #nm
d_px = 0.5
d = np.abs(H_a-D_a)
d

0.17830000000003565

# Análisis

calibración

In [54]:
np.abs(Ne_-H_a)

2.991260000000011

In [55]:
# Para H y 2H
def px_to_nm(px):
    return px*(np.abs(Ne_-H_a)/d_Ne_H)

d_ = px_to_nm(d_H_D)
d_

0.1774969455252925

In [56]:
Error_relativo(d,d_)

0.004503951064178342

In [57]:
# Desviación estandard de d_
SD_nm = ((np.abs(Ne_-H_a)*(d_px))/d_Ne_H)*np.sqrt(1+(1/((d_H_D*d_Ne_H)**2)))
SD_nm

0.001454892996293948

In [58]:
presicion(SD_nm,d_)

(0.008196721312517643, True)

In [59]:
exactitud(d,d_,SD_nm)

(0.0008030544747431589, True)

constante de proporcionalidad entre protón y electrón: mp = a*me, mp=mn

In [60]:
a = ((36*h*c)/(10*Ry*d_))
a

1848.7193007128444

In [65]:
a_r = (mp+mn)/(2*me)
a_r

1837.418167571577

In [66]:
Error_relativo(a_r,a)

0.006150550452107245

In [62]:
SD_a = ((36*h*c)/(10*Ry))*(1/(d_**2))*(SD_nm)
SD_a

15.153436893015686

In [63]:
presicion(SD_a,a)

(0.008196721312517643, True)

In [64]:
exactitud(a_r,a,SD_a)

(11.301133141267428, True)